In [1]:
import gym

import torch.nn.functional as F
from rrc_simulation.gym_wrapper.envs import cube_env
from spinup import ppo_pytorch as ppo
from gym.wrappers import FlattenObservation, FilterObservation, RescaleAction, ClipAction

In [ ]:
env_fn = lambda : ClipAction(RescaleAction(FlattenObservation(FilterObservation(
    cube_env.FlattenDictWrapper(gym.make('rrc_simulation.gym_wrapper:real_robot_challenge_phase_1-v1')),
    filter_keys=['desired_goal', 'observation'])), -1, 1))

ac_kwargs = dict(hidden_sizes=[64,64], activation=nn.Tanh)

logger_kwargs = dict(output_dir='./data/ppo_rrc_cube', exp_name='cube_rrc')

ppo(env_fn=env_fn, ac_kwargs=ac_kwargs, steps_per_epoch=5000, epochs=250, logger_kwargs=logger_kwargs)

Logging data to ./data/ppo_rrc_cube/progress.txt
Saving config:

{
    "ac_kwargs":	{
        "activation":	"Tanh",
        "hidden_sizes":	[
            64,
            64
        ]
    },
    "actor_critic":	"MLPActorCritic",
    "clip_ratio":	0.2,
    "env_fn":	"<function <lambda> at 0x7fcb3c721b70>",
    "epochs":	250,
    "exp_name":	"cube_rrc",
    "gamma":	0.99,
    "lam":	0.97,
    "logger":	{
        "<spinup.utils.logx.EpochLogger object at 0x7fcb3c6eec88>":	{
            "epoch_dict":	{},
            "exp_name":	"cube_rrc",
            "first_row":	true,
            "log_current_row":	{},
            "log_headers":	[],
            "output_dir":	"./data/ppo_rrc_cube",
            "output_file":	{
                "<_io.TextIOWrapper name='./data/ppo_rrc_cube/progress.txt' mode='w' encoding='UTF-8'>":	{
                    "mode":	"w"
                }
            }
        }
    },
    "logger_kwargs":	{
        "exp_name":	"cube_rrc",
        "output_dir":	"./data/ppo_rrc_cub

/scr1/.pyenv/versions/miniconda3-latest/envs/rrc/lib/python3.6/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
